In [1]:
from darli.modeling import Robot
from darli.modeling.functional import Functional
from darli.backend import PinocchioBackend, CasadiBackend
from robot_descriptions import z1_description
import numpy as np

In [2]:
robot = Robot(PinocchioBackend(z1_description.URDF_PATH))
robot.add_body({"ee": "link06", "mid": "link03"})

funct = Functional(CasadiBackend(z1_description.URDF_PATH))
funct.add_body({"ee": "link06", "mid": "link03"})

In [3]:
def compute_pin(robot: Robot):
    q, dq, ddq = np.random.rand(3, robot.nq)

    robot.update(q, dq, ddq)

    return {
        "com_pos": robot.com(),
        "kinetic": robot.energy().kinetic,
        "body1_pos": robot.body("ee").position,
        "body1_rot": robot.body("ee").rotation,
        "body2_pos": robot.body("mid").position,
        "body2_rot": robot.body("mid").rotation,
    }


def compute_casadi(funct: Functional):
    q, dq, ddq = np.random.rand(3, funct.nq)

    return {
        "com_pos": funct.com.position(q),
        "kinetic": funct.energy.kinetic(q, dq),
        "body1_pos": funct.body("ee").position(q),
        "body1_rot": funct.body("ee").rotation(q),
        "body2_pos": funct.body("mid").position(q),
        "body2_rot": funct.body("mid").rotation(q),
    }

In [4]:
compute_pin(robot)

/home/leo/github.com/lvjonok/darli/darli/backend/pinocchio.py:254: UserWarning: time variation of CoM jacobian is not implemented for pinocchio backend
  warnings.warn(


{'com_pos': CoM(position=array([-0.07578082, -0.06598864,  0.15864259]), velocity=array([-0.01733398, -0.08430195,  0.02152768]), acceleration=array([ 0.06824775, -0.08916896,  0.18832031]), jacobian=array([[ 6.59886414e-02,  4.39509415e-02, -8.46198804e-02,
         -1.96452920e-02, -5.43433363e-03, -7.37288085e-05],
        [-7.57808236e-02,  3.91617365e-02, -7.53991005e-02,
         -1.75046022e-02,  6.37717783e-03,  7.47404924e-05],
        [ 0.00000000e+00,  1.00478883e-01, -3.29804655e-02,
          4.21915673e-03,  1.07951973e-03,  1.49226790e-05]]), jacobian_dt=None),
 'kinetic': 0.067122312101382,
 'body1_pos': array([-0.0128,  0.    ,  0.1605]),
 'body1_rot': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'body2_pos': array([-0.35  ,  0.    ,  0.1035]),
 'body2_rot': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])}

In [5]:
compute_casadi(funct)

{'com_pos': DM([-0.11436, -0.069213, 0.0558207]),
 'kinetic': DM(0.0296616),
 'body1_pos': DM([-0.138069, -0.0654431, -0.0827481]),
 'body1_rot': DM(
 [[-0.178216, -0.161654, 0.970622], 
  [0.353201, 0.910167, 0.216436], 
  [-0.918416, 0.381397, -0.10511]]),
 'body2_pos': DM([-0.291647, -0.186808, 0.153948]),
 'body2_rot': DM(
 [[0.570989, -0.539369, 0.618913], 
  [0.365734, 0.84207, 0.396431], 
  [-0.73499, 0, 0.678078]])}

In [6]:
%timeit compute_pin(robot)

185 µs ± 21.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%timeit compute_casadi(funct)

273 µs ± 41.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
